In [8]:
import Pkg; Pkg.activate(@__DIR__);
ENV["CONDA_JL_HOME"] = raw"C:\Conda-Julia\3";Pkg.build("Conda");
Pkg.instantiate();
Pkg.build("PyCall")


  Activating project at `f:\Programming\Julia\CMU_OptimalControl\LectureNotes\CMU_optimal_control_lecture-notebooks\Lecture 7`
    Building Conda → `C:\Users\Amir Saman\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\51cab8e982c5b598eea9c8ceaced4b58d9dd37c9\build.log`
    Building Conda ─→ `C:\Users\Amir Saman\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\51cab8e982c5b598eea9c8ceaced4b58d9dd37c9\build.log`
    Building PyCall → `C:\Users\Amir Saman\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9816a3826b0ebf49ab4926e2b18842ad8b5c8f04\build.log`


In [9]:
using LinearAlgebra
using PyPlot

┌ Info: Installing matplotlib via the Conda matplotlib package...
└ @ PyCall C:\Users\Amir Saman\.julia\packages\PyCall\1gn3u\src\PyCall.jl:719
┌ Info: Running `conda install -y matplotlib` in root environment
└ @ Conda C:\Users\Amir Saman\.julia\packages\Conda\sDjAP\src\Conda.jl:181


Channels:
 - conda-forge
Platform: win-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 24.1.2
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: C:\Conda-Julia\3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-1.1.0               |       hcfcfb64_1          19 KB  conda-forge
    brotli-bin-1.1.0           |       hcfcfb64_1          20 KB  conda-forge
    contourpy-1.2.0            |  py310h232114e_0         186 KB  conda-forge
    cycler-0.12.1              |     pyhd8ed1ab_0          13 KB  conda-forge
    fonttools-4.50.0           |  py310h8d17308_0         1.8 MB  conda-forge
    freetype-2.12.1            |       hdaf720e_2         498 KB  conda-forge
    gettext-0.21.1             |       h5728263_0         5.3 MB  conda-forge
    glib-2.80.0                |       h39d0aa6_1         480 KB  conda-forge
    glib-tools-2.80.0          |       h0a98069_1          92 KB  conda-forge
    gst-plugins-base-1.22.9    |       h001b923

In [41]:
# getting discrete dynamics from continuous dynamics using ZOH
# Continuous dynamics
A = [0 1; 0 0]
B = [0.; 1]

# Define the sampling time and the number of subintervals
Ts = 0.1
N = 1000

# Compute the length of each subinterval
h = Ts/N

# Initialize Bd as a zero matrix
Bd = zeros(size(B))

# Compute Bd using the trapezoidal rule
for i in 0:N
    t = i*h
    Bd += exp(A*t)*B*h
end

# Discretize A
Ad = exp(A*Ts)

@show Ad
@show Bd



# Discrete dynamics (raw)
h = 0.1   # time step
A = [1 h; 0 1]
B = [0.5*h*h; h]


@show A
@show B

Ad = [1.0 0.1; 0.0 1.0]
Bd = [0.005005000000000001, 0.10010000000000184]
A = [1.0 0.1; 0.0 1.0]
B = [0.005000000000000001, 0.1]


2-element Vector{Float64}:
 0.005000000000000001
 0.1

In [ ]:
n = 2     # number of state
m = 1     # number of controls
Tfinal = 10.0 # final time #try larger values
N = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(N-1), step=h));

In [ ]:
# Initial conditions
x0 = [1.0; 0]

In [ ]:
# Cost weights
Q = 1.0*I(2)
R = 0.1
Qn = 1.0*I(2)

In [ ]:
function J(xhist,uhist)
    cost = 0.5*xhist[:,end]'*Qn*xhist[:,end]
    for k = 1:(N-1)
        cost = cost + 0.5*xhist[:,k]'*Q*xhist[:,k] + 0.5*uhist[k]'*R*uhist[k]
    end
    return cost
end

In [ ]:
function rollout(xhist, uhist)
    xnew = zeros(size(xhist))
    xnew[:,1] = xhist[:,1]
    for k = 1:(N-1)
        xnew[:,k+1] .= A*xnew[:,k] + B*uhist[k]
    end
    return xnew
end

In [ ]:
# Initial guess
xhist = repeat(x0, 1, N)
uhist = zeros(N-1)
Δu = ones(N-1)
λhist = zeros(n,N)

xhist = rollout(xhist, uhist) #initial rollout to get state trajectory

J(xhist,uhist) #Initial cost

In [ ]:
b = 1e-2 #line search tolerance
α = 1.0
iter = 0
while maximum(abs.(Δu[:])) > 1e-2 #terminate when the gradient is small
    
    #Backward pass to compute λ and Δu
    λhist[:,N] .= Qn*xhist[:,N]
    for k = N-1:-1:1
        Δu[k] = -(uhist[k]+R\B'*λhist[:,k+1])
        λhist[:,k] .= Q*xhist[:,k] + A'*λhist[:,k+1]
    end
    
    #Forward pass with line search to compute x
    α = 1.0
    unew = uhist + α.*Δu
    xnew = rollout(xhist, unew)
    while J(xnew, unew) > J(xhist, uhist) - b*α*Δu[:]'*Δu[:]
        α = 0.5*α
        unew = uhist + α.*Δu
        xnew = rollout(xhist, unew)
    end
    uhist .= unew;
    xhist .= xnew;
    iter += 1
end

In [ ]:
iter

In [ ]:
J(xhist,uhist) #cost of solution

In [ ]:
# Plot x1 vs. x2, u vs. t, x vs. t, etc.
plot(thist,xhist[1,:], label="Position")
plot(thist,xhist[2,:], label="Velocity")
xlabel("Time")
legend()

In [ ]:
plot(thist[1:end-1], uhist, label="control")
xlabel("Time")
legend()